In [235]:
#
#import numpy as np
#import tensorflow as tf
#import pandas as pd
#from random import shuffle
#from keras import Sequential,layers,losses,optimizers

In [236]:
N=4

In [237]:
with open('brown.txt','r') as eng, open('lalka-tom-pierwszy.txt','r',encoding='utf8') as pol:
    ang = {i.lower() for i in eng.read().split() if i.isalpha()}
    pol = {i.lower() for i in pol.read().split() if i.isalpha()}
    words = ang | pol

In [238]:
from itertools import chain
def N_gram(word,n:int=2):
    tokens = [token for token in word]
    n_grams = zip(*[tokens[i:] for i in range(n)])
    n_grams = [''.join(i) for i in n_grams]
    return n_grams

def every_N_gram(word,n:int=2):
    return list(chain.from_iterable([N_gram(word,i) for i in range(1,n+1)]))

def buildable_n_gram(words_list,n=2):
    t = set()
    for word in words_list:
        for gram in every_N_gram(word,n):
            t.add(gram)
    return t

In [239]:
every_n_gram_list = buildable_n_gram(words,N)
print(len(every_n_gram_list))

49348


Dla każdego słowa można znalezc n-gramy
obliczyc n-gramy polskie i angielskie osobno, porównać częstotliwość i wybrać to większe

In [240]:
def lang_gram_freq(words,n=2):
    lang_freq = {}
    for word in words:
        grams = every_N_gram(word,n)
        for gram in grams:
            if gram in lang_freq:
                lang_freq[gram] += 1
            else:
                lang_freq[gram] = 1
    return {gr:lang_freq[gr]/len(words) for gr in lang_freq}
pol_freq = lang_gram_freq(pol,N)
eng_freq = lang_gram_freq(ang,N)

In [241]:
def freq(dct,gram):
    return dct[gram] if gram in dct else 0

In [242]:
languages_freq_compare = {gram:tuple(map(lambda x:freq(x,gram),[pol_freq,eng_freq]))for gram in every_n_gram_list}

In [243]:
def word_score(word,n=2):
    score = 0
    for t in every_N_gram(word,n):
        if t not in languages_freq_compare:
            continue
        p,e = languages_freq_compare[t]
        if p == 0:
            return -1
        if e == 0:
            return 1
        else:
            if p == e:continue
            score += p/e if p>e else -p/e
    return score
        
def guess_lang(word,n=2):
    return 'pol' if word_score(word,n) >= 0 else 'ang'

In [244]:
#N=5 0.9316992830846735

In [245]:
with open('do_wprawki.txt','r',encoding='utf8') as data:
    data = data.read().splitlines()
    test_set = {word.split()[1]:word.split()[0] for word in data}
    ans = [1 for i in test_set if guess_lang(i,N)==test_set[i]]
    print(len(ans)/len(test_set))

0.9278240651036621


In [ ]:
for i in test_set:
    if guess_lang(i,N)!=test_set[i]:
        print(i)